# model_utils.py

Creamos un método que encapsule el bucle de selección de proyectos tomando como parámetros de entrada las mutaciones y el dataframe.

In [2]:
# Librerías

import pandas as pd
import random

# Datos de prueba

data = {
    'MATRICULA_DIGITAL': ['Proyecto B ETAPA 2', 'Proyecto A ETAPA 2', 'Proyecto A ETAPA 1', 'Proyecto C', 'Proyecto B ETAPA 1','Proyecto X','Proyecto Y','Proyecto Z','Proyecto AA','Proyecto AB'],
    'DEPENDENCIAS': ['Proyecto B ETAPA 1, Proyecto C', 'Proyecto A ETAPA 1', '', '', 'Proyecto A ETAPA 2','','','','',''],
    'EXCLUYENTES': ['', '', '', 'Proyecto A ETAPA 1, Proyecto A ETAPA 2', '','','','Proyecto AA, Proyecto AB','Proyecto Z, Proyecto AB','Proyecto Z, Proyecto AA'],
}

df_projects = pd.DataFrame(data)
mutants = [1,1,1,1,1,1,0,1,1,0]

In [3]:
# Creación del método en el model_utils.py (debajo de la restricción de check_capex)

def seleccionar_proyectos_excluidos(mutants, df_projects):
    """
    Selects projects considering their dependencies and exclusions.

    This function takes a DataFrame containing project information as input and selects projects that can be executed considering 
    their dependencies and exclusions.
    
    :param mutant: np.array containing the portafolio to be checked
    :type mutant: np.array
    :param df_projects: df containing projects
    :type df_projects: DataFrame

    :return: Tuple containing lists of projects that are available and excluded for selection based on their dependencies and exclusions.
    :rtype: tuple
    """
    available_projects = []
    excluded_projects = []
    lista_pro_exl_ori = df_projects["MATRICULA_DIGITAL"].unique().tolist()
    lista_pro_exl = lista_pro_exl_ori.copy()

    for i in range(len(mutants)):
        if mutants[i] == 1:
            project = df_projects.loc[i, 'MATRICULA_DIGITAL']
            exclusions = [ex.strip() for ex in df_projects.loc[i, 'EXCLUYENTES'].split(',')]

            if all(ex in available_projects + excluded_projects for ex in exclusions):
                excluded = False
                for ex in exclusions:
                    if ex in available_projects:
                        excluded = True
                        break
                lista_pro_exl.remove(project)
                if excluded:
                    excluded_projects.append(project)
                else:
                    available_projects.append(project)
            else:
                pass

    i = 0
    n = len(lista_pro_exl)
    while len(lista_pro_exl) > 0:
        i += 1
        if i > n:
            break
        df_trabajo = df_projects[df_projects["MATRICULA_DIGITAL"].isin(lista_pro_exl)]
        for index, row in df_trabajo.iterrows():
            project = row['MATRICULA_DIGITAL']
            exclusions = [ex.strip() for ex in row['EXCLUYENTES'].split(',')]
            
            es_aleatorio = False
            lista_aleatorio = [project]
            for ex in exclusions:
                if ex in lista_pro_exl_ori:
                    exclusiones_ex = [ex.strip() for ex in df_trabajo["EXCLUYENTES"][df_trabajo["MATRICULA_DIGITAL"]==ex].values[0].split(',')]
                    if project in exclusiones_ex:
                        es_aleatorio = True
                        lista_aleatorio.append(ex)
            
            if es_aleatorio:
                pro_sel_ale = random.choice(lista_aleatorio)
                lista_pro_exl.remove(pro_sel_ale)
                lista_aleatorio.remove(pro_sel_ale)
                available_projects.append(pro_sel_ale)
                excluded_projects.extend(lista_aleatorio)
                for pro in lista_aleatorio:
                    lista_pro_exl.remove(pro)
                break
            else:
                if all(ex in available_projects + excluded_projects for ex in exclusions):
                    excluded = False
                    for ex in exclusions:
                        if ex in available_projects:
                            excluded = True
                            break
                    lista_pro_exl.remove(project)
                    if excluded:
                        excluded_projects.append(project)
                    else:
                        available_projects.append(project)
                else:
                    pass

    return available_projects, excluded_projects

In [4]:
# Verificación

available_projects, excluded_projects = seleccionar_proyectos_excluidos(mutants, df_projects)
print("Proyectos disponibles:", available_projects)
print("Proyectos excluidos:", excluded_projects)

Proyectos disponibles: ['Proyecto AB']
Proyectos excluidos: ['Proyecto Z', 'Proyecto AA']


## model.py

Sabiendo que la metaheurística (algoritmo genético) se crea directamente en el método **init_pygad(self)**, podemos bajar al método 
**mutation(offsprings, ga_instance)** y colocar el método definido anteriormente de la siguiente manera:

In [ ]:
        def mutation(offsprings, ga_instance):
            """
            Method to perform mutation on the offspring solutions.
            include the contraceptive method to ensure no over-budget (capex restriction)

            :param offsprings: A 2D NumPy array representing the offspring solutions.
            :type offsprings: numpy.ndarray

            :param ga_instance: pygad instance containing the parameters and configuration.
            :type ga_instance: pygad instance

            :return: A new 2D NumPy array containing the mutated offspring solutions.
            :rtype: numpy.ndarray

            """

            if len(offsprings.shape) != 2:
                raise ValueError("offsprings array must be 2-dimensional.")

            mutants = offsprings
            mutation_rate = self.pb_parameters["mutation_ind"]
            gen_mutation_rate = self.pb_parameters["mutation_gen"]
            for i in range(mutants.shape[0]):
                random_value = rd.random()
                if random_value <= mutation_rate:
                    # int_random_value = rd.randint(0, mutants[i].shape[0] - 1)
                    rand_arr = np.random.rand(len(mutants[i]))
                    genes_mutate = np.where(rand_arr < gen_mutation_rate)[0]
                    genes_0 = np.where(mutants[i] == 0)[0]
                    genes_1 = np.where(mutants[i] == 1)[0]

                    genes_0_mutate = genes_0[np.in1d(genes_0, genes_mutate)]
                    genes_1_mutate = genes_1[np.in1d(genes_1, genes_mutate)]

                    mutants[i, genes_0_mutate] = 1
                    mutants[i, genes_1_mutate] = 0

                # turn off projects with basic off
                actives_off = get_off_actives(self.df_projects, mutants[i])
                indexes_turn_off = get_indexes_turn_off(self.df_projects, actives_off)
                mutants[i][indexes_turn_off] = 0

                #check ISA restriction
                check_isa(mutants[i], self.indexes_isa)

                #check capex restriction
                check_capex(mutants[i], self.df_projects, self.df_semi_mandatory, self.mandatory_capex, actives_off, self.config)

                #check exclusions and dependencies restriction --> Acá colocamos la restricción que recibiría como parámetros de entrada las mutaciones y el df de proyectos con sus respectivas exclusiones y dependencias
                seleccionar_proyectos_excluidos(mutants[i], self.df_projects)

            return mutants

## ¡Gracias!